In [1]:
from pymopt.solid import SolidPlateModel

from scipy import stats
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks", {'grid.linestyle': '--'})
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def radialDistance(p,w,nn,dr):
    alpha = np.array([(i)*dr for i in range(nn+1)])
    da = np.array([2*np.pi*(i+0.5)*dr**2 for i in range(nn)])
    r = np.sqrt(p[0]**2+p[1]**2)
    Rdr = []
    for i in range(nn):
        index = np.where((alpha[i]<r)&(alpha[i+1]>=r))[0]
        Rdr.append(w[index].sum())
    Rdr = np.array(Rdr)/(da*nPh)
    return alpha[:-1],Rdr

def AngularyResolved(v,w,nn):
    da = np.pi/(2*nn)
    alpha = np.array([(i+0.5)*da for i in range(nn+1)])
    alpha2 = np.array([(i)*da for i in range(nn+1)])
    do = 4*np.pi*np.sin(alpha)*np.sin(da/2)
    at = np.arccos(np.sign(v[2])*(v[2]))
    Rda = []
    for i in range(nn):
        index = np.where((alpha2[i]<at)&(alpha2[i+1]>=at))[0]
        Rda.append(w[index].sum())
    Rda = np.array(Rda)/(do[:-1]*nPh)
    return alpha[:-1],Rda

# Computation Results of single-solidmodel and mcml

## Total diffuse reflectance and total transmittance

### slab of turbid medium

In [3]:
nPh = 500000
model = SolidPlateModel(nPh = nPh)
params = {
    'thickness':[0.02],
    'ms':[90],
    'ma':[10],
    'g':[0.75],
    'n':[1.],
    'n_air':1.,
}
model.built(**params)
model = model.start()



###### Start ######

Progress: 36.021 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 70.153 [％]
Calculation time: 0 minutes 1 seconds.

Progress: 91.818 [％]
Calculation time: 0 minutes 1 seconds.

Progress: 99.182 [％]
Calculation time: 0 minutes 1 seconds.

Progress: 99.992 [％]
Calculation time: 0 minutes 1 seconds.


###### Finish ######
Maximum step number: 44
######
Mean Rd 0.097505
Mean Tt 0.660834

Calculation time: 0 minutes 1 seconds.


### Semi-infinite turbid medium that has mismatched refractive index with the ambient medium

In [3]:
nPh = 100000
model = SolidPlateModel(nPh = nPh)
params = {
    'thickness':[10],
    'ms':[90],
    'ma':[10],
    'g':[0.],
    'n':[1.5],
    'n_air':1.,
}
model.built(**params)
model = model.start()

## Angularly resolved diffuse reflectance and transmittance

In [ ]:
nPh = 500000
model = SolidPlateModel(nPh = nPh)
params = {
    'thickness':[0.02],
    'ms':[90],
    'ma':[10],
    'g':[0.75],
    'n':[1.],
    'n_air':1.,
}
model.built(**params)
model = model.start()

In [ ]:
rez = model.getResult()
v_result = rez["v"]
w_result = rez["w"]
Tt_index = np.where(v_result[2]>0)[0]
Rd_index = np.where(v_result[2]<0)[0]

In [ ]:
print('Trancemittance')
nn = 30
alpha,Tta = AngularyResolved(v_result[:,Tt_index],w_result[Tt_index],nn)

plt.figure(figsize=(5,4),dpi=90)
plt.plot(alpha/np.pi,Tta,'.')
plt.xlabel('Exit Angle α [rad/π]')
plt.ylabel('Tt(α) [/sr]')
plt.xlim(0,0.5)
plt.ylim(0,0.8)
plt.show()

In [ ]:
print('Refrected')
nn = 30
alpha,Rda = AngularyResolved(v_result[:,Rd_index],w_result[Rd_index],nn)

plt.figure(figsize=(5,4),dpi=90)
plt.plot(alpha/np.pi,Rda,'.')
plt.xlabel('Exit Angle α [rad/π]')
plt.ylabel('Rd(α) [/sr]')
#plt.xlim(0,0.5)
#plt.ylim(0,0.025)
plt.show()

In [ ]:
df_ = pa.DataFrame(np.array([alpha/np.pi,Rda,Tta]).T,columns = ['Angle','Rd','Tt'])
df_.to_csv('result/solid_angle.csv',index=None)

## Radially resolved diffuse reflectance

In [ ]:
nPh = 50000

params = {
    'thickness':[20],
    'ms':[100],
    'ma':[0.1],
    'g':[0.9],
    'n':[1.],
    'n_air':1.,
}
model = SolidPlateModel(nPh = nPh)
model.built(**params)
model = model.start()

In [ ]:

params = {
    'thickness':[20],
    'ms':[10],
    'ma':[0.1],
    'g':[0.],
    'n':[1.],
    'n_air':1.,
}
model = SolidPlateModel(nPh = nPh)
model.built(**params)
model = model.start()

In [ ]:
nn = 100
dr = 0.01

rezB = model.getResult()
p_resultB = rezB["p"]
v_resultB = rezB["v"]
w_resultB = rezB["w"]
Tt_indexB = np.where(v_resultB[2]>0)[0]
Rd_indexB = np.where(v_resultB[2]<0)[0]

alphaA,RdrA = radialDistance(p_resultA[:,Rd_indexA],w_resultA[Rd_indexA],nn,dr)
alphaB,RdrB = radialDistance(p_resultB[:,Rd_indexB],w_resultB[Rd_indexB],nn,dr)

In [ ]:

plt.figure(figsize=(5,4),dpi=90)
plt.plot(alphaA,RdrA,'-',c = 'red',label = 'A: g = 0.9')
plt.plot(alphaB,RdrB,'-',c = 'blue',label = "B: g = 0.0")
plt.yscale('log')
plt.xlim(0,1)
plt.legend()
plt.xlabel('r [cm]')
plt.ylabel('Rd $[cm^{-2}]$')
plt.show()

In [ ]:
plt.figure(figsize=(5,4),dpi=90)
plt.plot(alphaA,(RdrB-RdrA)/RdrA,'-',c = 'red',label = '(B-A) / A')
plt.xlim(0,1)
plt.ylim(-0.5,2)
plt.legend()
plt.xlabel('r [cm]')
plt.ylabel('Relative Error')
plt.show()

In [ ]:
df_ = pa.DataFrame(np.array([alphaA,RdrA,RdrB]).T,columns = ['r cm','A_Rd','B_Rd'])
df_.to_csv('result/solid_equivalent.csv',index=None)

## Depth resolved internal fluence

In [2]:
model_params = {
    'nPh': 1000,
    'fluence': True,
    'nr': 500,
    'nz': 200,
    'dr': 0.01,
    'dz': 0.01,
}

print("######  n = 1.  ######")
params = {
    'thickness':[200],
    'ms':[100],
    'ma':[0.1],
    'g':[0.9],
    'n':[1.],
    'n_air':1.,
}
model = SolidPlateModel(**model_params)
model.built(**params)
model = model.start()
fluenceA = model.getFluence()

print('')
print("######  n = 1.37  ######")
params = {
    'thickness':[200],
    'ms':[100],
    'ma':[0.1],
    'g':[0.9],
    'n':[1.37],
    'n_air':1.,
}
model = SolidPlateModel(**model_params)
model.built(**params)
model = model.start()
fluenceB = model.getFluence()

######  n = 1.  ######
Memory area size for fluence storage: 0 Mbyte

###### Start ######

Progress: 0.6 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 1.3 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 4.1 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 10.5 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 23.4 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 43.6 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 58.1 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 69.4 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 77.6 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 83.1 [％]
Calculation time: 0 minutes 0 seconds.

Progress: 87.9 [％]
Calculation time: 0 minutes 1 seconds.

Progress: 91.9 [％]
Calculation time: 0 minutes 2 seconds.

Progress: 94.3 [％]
Calculation time: 0 minutes 4 seconds.


###### Finish ######
Maximum step number: 13823
######
Mean Rd 0.749548
Mean Tt 0.000000

Calculation time: 0 minutes 6 seconds.
######  n = 1.3

In [ ]:
da = np.array([2*np.pi*(i+0.5)*model_params['dr']**2 for i in range(model_params['nr'])])
aaa = params['ma'][0]*model_params['nPh']*model_params['dz']
phi_zA = (fluenceA["Arz"].T/aaa).T.sum(0)
phi_zB = (fluenceB["Arz"].T/aaa).T.sum(0)

plt.figure(figsize=(5,4),dpi=100)
plt.plot(fluenceA["z"],phi_zA,c = 'red',label = 'n = 1.0')
plt.plot(fluenceB["z"],phi_zB,c = 'blue',label = 'n = 1.37')
plt.xlabel('Z [cm]')
plt.ylabel('Fluence [-]')
plt.legend()
plt.yscale('log')
plt.xlim(0,1)
plt.ylim(0.5,10)
plt.show()

In [ ]:
df_ = pa.DataFrame(np.array([fluence.getArrayZ(),phi_zA,phi_zB]).T,columns = ['z cm','n_1','n_1.37'])
df_.to_csv('result/voxel_fluence_z.csv',index=None)

# Multi-layered Tissues

## Refrective and transmittnace rate

In [ ]:
nPh = 100000
params = {
    'thickness':[0.1,0.1,0.2],
    'ms':[100,10,10],
    'ma':[1,1,2],
    'g':[0.9,0,0.7],
    'n':[1.37,1.37,1.37],
    'n_air':1.,
}
model = SolidPlateModel(nPh = nPh)
model.built(**params)
model = model.start()

In [ ]:
rez = model.getResult()
p_result = rez["p"]
v_result = rez["v"]
w_result = rez["w"]

Tt_index = np.where(v_result[2]>0)[0]
Rd_index = np.where(v_result[2]<0)[0]

In [ ]:
nn = 100
dr = 0.005

alphaRd,Rd = radialDistance(p_result[:,Rd_index],w_result[Rd_index],nn,dr)
alphaTt,Tt = radialDistance(p_result[:,Tt_index],w_result[Tt_index],nn,dr)

plt.figure(figsize=(5,4),dpi=100)
plt.plot(alphaRd,Rd,'-',c = 'red')
plt.yscale('log')
plt.xlim(0,0.5)
plt.ylim(10**-2,10**2)
plt.xlabel('r [cm]')
plt.ylabel('Rd $[cm^{-2}]$')
plt.show()

plt.figure(figsize=(5,4),dpi=100)
plt.plot(alphaTt,Tt,'-',c = 'blue')
plt.yscale('log')
plt.xlim(0,0.5)
plt.ylim(10**-2,1)
plt.xlabel('r [cm]')
plt.ylabel('Tt $[cm^{-2}]$')
plt.show()

In [ ]:
df_ = pa.DataFrame(np.array([alphaRd,Rd,Tt]).T,columns = ['r cm','Rd','Tt'])
df_.to_csv('result/solid_multi_rt_test.csv',index=None)